In [1]:
from detr_config import Config
from detr_trainer import get_trainer
from detr_model_evaluator import ModelEvaluator
from detr_model_loader import ModelLoader
from detr_file_manager import FileManager
from detr_metrics import MetricsAggregator
from detr_model_loader import ModelLoader
from detr_factory import DETRFactory
from detr_data_source import DataSource

In [2]:

Config.set_local_settings()
# Config.set_cpu_settings()
# Config.set_gpu_settings()

for architecture, num_queries, transformer_layers in Config.HYPERPARAMS:
    
    detr_factory = DETRFactory(architecture, num_queries, transformer_layers)
    image_processor = detr_factory.new_image_processor()
    
    model_name = detr_factory.get_model_name()
    file_manager = FileManager(model_name)
    
    data_source = DataSource(image_processor, file_manager)
    model_loader = ModelLoader(detr_factory, file_manager)
    
    file_manager.clean_model_logs()
        
    # -----------------------
    # K-fold Cross Validation 
    # -----------------------
    
    metrics_aggregator = MetricsAggregator(file_manager)

    for fold in range(1,11):

        # Model
        
        model = model_loader.new_pretrained_model()
        
        # Dataset
        
        file_manager.set_validation_fold_setup(fold)
        
        train_dataset, train_loader = data_source.get_train_dataset_dataloader()
        valid_dataset, valid_loader = data_source.get_valid_dataset_dataloader()

        # Training
        
        fold_version = file_manager.get_fold_version()
        trainer = get_trainer(fold_version)
        trainer.fit(model, train_loader, valid_loader)
        
        # Validation
        
        best_model = model_loader.load_best_model()
        model_evaluator = ModelEvaluator(model, image_processor, valid_dataset, valid_loader)
        valid_metrics = model_evaluator.get_metrics(threshold=Config.THRESHOLD)
        
        metrics_aggregator.add_valid_metrics(valid_metrics)
        metrics_aggregator.save_metrics()
        
        file_manager.clean_checkpoints()

        # break # Fold
    
    # Aggregate K-Fold Metrics
    
    metrics_aggregator.calculate_valid_mean()
    metrics_aggregator.save_metrics()
    
    # -----------------------
    # Testing
    # -----------------------
    
    # Model
    
    model = model_loader.new_pretrained_model()

    # Dataset
    
    file_manager.set_testing_setup()

    train_valid_dataset, train_valid_loader = data_source.get_train_valid_dataset_dataloader()
    test_dataset, test_loader = data_source.get_test_dataset_dataloader()
    
    # Training
    
    test_version = file_manager.get_test_version()
    trainer = get_trainer(test_version)
    trainer.fit(model, train_valid_loader, test_loader)
    
    # Testing

    best_model = model_loader.load_best_model()
    model_evaluator = ModelEvaluator(best_model, image_processor, test_dataset, test_loader)
    test_metrics = model_evaluator.get_metrics(threshold=Config.THRESHOLD)
    
    # Metrics
    
    metrics_aggregator.add_test_metrics(test_metrics)
    metrics_aggregator.save_metrics()
    
    # break # Hyperparameter

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


New Model:  model=DETR_queries=100_layers=6


Some weights of DetrForObjectDetection were not initialized from the model checkpoint at facebook/detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([92, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading Annotations from:  InBreast_Coco\fold_1\train\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading Annotations from:  InBreast_Coco\fold_1\valid\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type                   | Params
------------------------------------------------
0 | detr | DetrForObjectDetection | 41.5 M
------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.007   Total estimated model params size (MB)


...

Sanity Checking: 0it [00:00, ?it/s]

c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:490: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which i

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


Loading:  lightning_logs\model=DETR_queries=100_layers=6\test\checkpoints\epoch=0-step=1.ckpt
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.021
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0